In [1]:
import os
from doctest import debug

import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import librosa

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sympy.codegen.cnodes import static
from torch.utils.data import Dataset, DataLoader

In [2]:
# data paths
mixture_directory_test = "data/mixture/val"
mixture_directory_train = "data/mixture/train"
vocal_directory_test = "data/vocal/val"
vocal_directory_train = "data/vocal/train"

In [3]:
# parameters to play around with to better optimize training
# torch.backends.cudnn.benchmark = True
patch_size = 128
stride = 64
batch_size = 1
epochs = 20
model_save_path = "vocal_isolator.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # will utilize GPU if possible to train/test

In [4]:
def complex_spec_loss(pred, target):
    """
    Compute loss in the frequency domain
    """
    # Convert to complex spectrograms - detach from computation graph first
    pred_mag = librosa.db_to_amplitude(pred.detach().cpu().numpy())
    target_mag = librosa.db_to_amplitude(target.detach().cpu().numpy())

    # Magnitude loss - convert back to tensor
    mag_loss = F.mse_loss(torch.tensor(pred_mag, device=pred.device),
                         torch.tensor(target_mag, device=target.device))

    return mag_loss

def custom_loss(pred, target, alpha=0.5):
    """
    Combined loss function
    """
    # Time-domain loss (MSE)
    mse_loss = F.mse_loss(pred, target)

    # Frequency-domain loss
    spec_loss = complex_spec_loss(pred, target)

    # Combine losses
    return mse_loss + alpha * spec_loss

In [5]:
class SpectrogramDataset(Dataset):
    def __init__(self, mixed_dir, vocal_dir, use_magnitude=True):
        self.mixed_map = self._make_file_map(mixed_dir, prefix='mix_')
        self.vocals_map = self._make_file_map(vocal_dir, prefix='vocal_')
        
        # error check with available song keys
        self.keys = sorted(list(set(self.mixed_map.keys()) & set(self.vocals_map.keys())))
        assert self.keys, "No matching files found between mixed and vocal directories"
        
        self.pairs = [(self.mixed_map[k], self.vocals_map[k]) for k in self.keys]
        self.use_magnitude = use_magnitude

    def augment_spectrogram(self, spec):
        # Convert to numpy for manipulation
        spec_np = spec.numpy()

        # Random frequency masking
        f = np.random.randint(0, 30)  # mask up to 30 frequency bins
        f0 = np.random.randint(0, spec_np.shape[1] - f)
        spec_np[:, f0:f0+f, :] = 0

        # Random time masking
        t = np.random.randint(0, 40)  # mask up to 40 time frames
        t0 = np.random.randint(0, spec_np.shape[2] - t)
        spec_np[:, :, t0:t0+t] = 0

        return torch.from_numpy(spec_np)


    @staticmethod
    def _make_file_map(directory, prefix):
        file_map = {}
        for path in glob.glob(os.path.join(directory, "*.npy")):
            filename = os.path.basename(path)
            if filename.startswith(prefix):
                key = filename[len(prefix):]  # strip the prefix
                file_map[key] = path
        return file_map
    
    # @staticmethod
    # def _load_excel_sheet(filepath):
    #     real = pd.read_excel(filepath, sheet_name='Re_X', header=None).values.astype(np.float32)
    #     imaginary = pd.read_excel(filepath, sheet_name='Im_X', header=None).values.astype(np.float32)
    #     fs = pd.read_excel(filepath, sheet_name='Sampling Rate', header=None).values[0][0]
    #     
    #     complex_stft = real + 1j*imaginary
    #     return complex_stft, fs
    
    @staticmethod
    def _load_npy(filepath):
        return np.load(filepath)
    
    @staticmethod
    def _pad_to_multiple_16(tensor, multiple=16):
        #Ensure input has 3 dimensions: [C, H, W]
        if tensor.ndim == 2:
            tensor = tensor.unsqueeze(0)  # Add channel dim for single-channel input

        _, h, w = tensor.shape
        pad_h = (multiple - h % multiple) % multiple
        pad_w = (multiple - w % multiple) % multiple
        return F.pad(tensor, (0, pad_w, 0, pad_h))
    
    def __len__(self):
        return len(self.pairs) #length of le dataset?BRO THIS IS WH Y IM NOT IN COMPUTER ENGINE
        
    def __getitem__(self, index):
        mixed_path, vocals_path = self.pairs[index]

        mixed_stft = self._load_npy(mixed_path)
        vocals_stft = self._load_npy(vocals_path)
        
        if self.use_magnitude:
            mixed_mag = np.abs(mixed_stft)
            vocals_mag = np.abs(vocals_stft)
            
            mixed_mag = (mixed_mag - mixed_mag.min()) / (mixed_mag.max() - mixed_mag.min() + 1e-8)
            vocals_mag = (vocals_mag - vocals_mag.min()) / (vocals_mag.max() - vocals_mag.min() + 1e-8)
            
            mixed_tensor = torch.tensor(mixed_mag)
            vocals_tensor = torch.tensor(vocals_mag)
            
            mixed_tensor = self._pad_to_multiple_16(mixed_tensor)
            vocals_tensor = self._pad_to_multiple_16(vocals_tensor)

            if np.random.rand() > 0.5:
                mixed_tensor = self.augment_spectrogram(mixed_tensor)
            
            return mixed_tensor, vocals_tensor

        else:  # if training on complex values
            return torch.tensor(mixed_stft), torch.tensor(vocals_stft)
        
def pad_collate_fn(batch):
    """
    Pads all spectrograms in the batch to the maximum height and width in that batch.
    """
    max_h = max(item[0].shape[1] for item in batch)
    max_w = max(item[0].shape[2] for item in batch)
    
    mixed_padded = []
    vocals_padded = []
    for mixed, vocals in batch:
        mixed_padded.append(F.pad(mixed, (0, max_w - mixed.shape[2], 0, max_h - mixed.shape[1])))
        vocals_padded.append(F.pad(vocals, (0, max_w - vocals.shape[2], 0, max_h - vocals.shape[1])))

    return torch.stack(mixed_padded), torch.stack(vocals_padded)


In [6]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels, dropout_rate=0.2):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout2d(dropout_rate),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True)
            )

        # Encoding
        self.encoder1 = conv_block(1, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = conv_block(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.center = conv_block(512, 1024, dropout_rate=0.3)

        # Decoding
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = conv_block(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = conv_block(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = conv_block(128, 64)

        self.final = nn.Conv2d(64, 1, kernel_size=1)  # Removed Sigmoid

    def forward(self, x):
        # encoding
        e1 = self.encoder1(x)
        e2 = self.encoder2(self.pool1(e1))
        e3 = self.encoder3(self.pool2(e2))
        e4 = self.encoder4(self.pool3(e3))
        center = self.center(self.pool4(e4))

        # decoding with skip connections
        d4 = self.dec4(torch.cat([self.up4(center), e4], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d4), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))

        return self.final(d1)

In [7]:
class VocalLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.L1Loss()

    def forward(self, pred, target):
        # L1 loss
        l1_loss = self.l1(pred, target)

        # Spectral loss
        pred_mag = torch.abs(torch.fft.rfft2(pred))
        target_mag = torch.abs(torch.fft.rfft2(target))
        spec_loss = self.l1(pred_mag, target_mag)

        # Combined loss
        total_loss = l1_loss + 0.1 * spec_loss
        return total_loss

In [8]:
# helper functions for training function
def compute_sdr(target, prediction):
    noise = target - prediction
    return 10 * torch.log10(torch.sum(target ** 2) / (torch.sum(noise ** 2) + 1e-8))

def save_prediction_as_npy(prediction, filename):
    prediction_np = prediction.squeeze().cpu().detach().numpy()
    np.save(filename, prediction_np)
    print(f"Saved prediction to {filename}")

In [9]:
    # training function to be called
def train(model, train_loader, val_loader, criterion, optimizer, scheduler, device, num_epochs, checkpoint_path="vocal_isolator.pth"):
    print("Starting training loop...")
    # os.makedirs(checkpoint_path, exist_ok=True)
    # model.train()
    # checkpoint_interval = 5
    model.to(device)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs} starting...")
        # TRAINING
        print("setting to training mode")
        model.train()  # set to training mode
        running_loss = 0.0
        # num_batches = 0
        debug_num = epoch+1

        for mixed, vocals in train_loader:
            # print("moved to GPU")
            # Move to GPU
            mixed = mixed.to(device)
            vocals = vocals.to(device)
            # print("forward pass starting")
            # Forward pass
            optimizer.zero_grad()
            # print("creating output from model")
            outputs = model(mixed)
            loss = criterion(outputs, vocals)
            
            # Backward pass
            # print("backward pass starting")
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            debug_num += 0.01
            # print(debug_num)

        # Calculate average loss
        train_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs} — Train Loss: {train_loss:.4f}")
        
        # VALIDATION
        print("validation beginning")
        model.eval()   # set to eval mode for validation
        val_loss, mae, sdr = 0.0, 0.0, 0.0
        with torch.no_grad():
            debug_num2 = epoch+1
            for mixed, vocals in val_loader:
                mixed, vocals = mixed.to(device), vocals.to(device)
                # print("getting output from mixed for validation")
                outputs = model(mixed)
                # print("calculating loss")
                val_loss += criterion(outputs, vocals).item()
                # print("calculating MAE")
                mae += F.l1_loss(outputs, vocals).item()
                # print("calculating SDR")
                sdr += compute_sdr(outputs, vocals)
                debug_num2 += 0.01
                # print(debug_num2)
        val_loss /= len(val_loader)
        mae /= len(val_loader)
        sdr /= len(val_loader)
        print(f"Validation: Loss = {val_loss:.4f}, MAE = {mae:.4f}, SDR = {sdr:.4f} dB")

        scheduler.step(val_loss)
        
        torch.save(model.state_dict(), checkpoint_path)
        
        # save predicted xlsx
        mixed_sample, _ = next(iter(val_loader))
        mixed_sample = mixed_sample.to(device)
        predicted = model(mixed_sample[:1])
        save_prediction_as_npy(predicted, f"predicted_epoch_{epoch}.npy")
    
    print(f"Training complete, model saved at {checkpoint_path}")
    return model

In [10]:
# training block
train_dataset = SpectrogramDataset(mixed_dir="data/mixture/train", vocal_dir="data/vocal/train")
val_dataset = SpectrogramDataset(mixed_dir="data/mixture/val", vocal_dir="data/vocal/val")

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn, num_workers=0)

print("Testing dataset loading...")
for i in range(min(3, len(train_dataset))):
    x,y = train_dataset[i]
    print(f"Sample {i} : x shape {x.shape}, y shape {y.shape}")

print(f"Training pairs: {len(train_dataset)}")
print(f"Validation pairs: {len(val_dataset)}")

model = UNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # load model onto device

# criterion = torch.nn.L1Loss()
# criterion = VocalLoss()
def custom_loss_wrapper():
    def loss_fn(pred, target):
        return custom_loss(pred, target, alpha=0.5)
    return loss_fn

criterion = custom_loss_wrapper()

optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3
)

train(
    model=model,
    train_loader=train_dataloader,
    val_loader=val_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    num_epochs=epochs,
    checkpoint_path='vocal_isolator.pth'
)

Testing dataset loading...
Sample 0 : x shape torch.Size([1, 528, 864]), y shape torch.Size([1, 528, 864])
Sample 1 : x shape torch.Size([1, 528, 864]), y shape torch.Size([1, 528, 864])
Sample 2 : x shape torch.Size([1, 528, 864]), y shape torch.Size([1, 528, 864])
Training pairs: 4426
Validation pairs: 2422
Starting training loop...
Epoch 1/20 starting...
setting to training mode
Epoch 1/20 — Train Loss: 0.0303
validation beginning
Validation: Loss = 0.0353, MAE = 0.1203, SDR = 3.6703 dB
Saved prediction to predicted_epoch_0.npy
Epoch 2/20 starting...
setting to training mode
Epoch 2/20 — Train Loss: 0.0290
validation beginning
Validation: Loss = 0.0426, MAE = 0.1228, SDR = 4.3067 dB
Saved prediction to predicted_epoch_1.npy
Epoch 3/20 starting...
setting to training mode
Epoch 3/20 — Train Loss: 0.0285
validation beginning
Validation: Loss = 0.0321, MAE = 0.1165, SDR = 4.6723 dB
Saved prediction to predicted_epoch_2.npy
Epoch 4/20 starting...
setting to training mode
Epoch 4/20 — Tr

UNet(
  (encoder1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Dropout2d(p=0.2, inplace=False)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Dropout2d(p=0.2, inplace=False)
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, mome